In [3]:
SAMPLES_DIR="../samples/"
ETC_DIR=SAMPLES_DIR+"etc/"
AUDIO_DIR=SAMPLES_DIR+"audio/"
OUT_DIR=SAMPLES_DIR+"out/"

In [4]:
VIDEO_NAME="video.mp4"
AUDIO_NAME="video.wav"

In [5]:
# Video length
from moviepy.editor import VideoFileClip

vid = VideoFileClip(SAMPLES_DIR+VIDEO_NAME)
duration = vid.duration

In [55]:
output = model.transcribe(AUDIO_DIR+"224.wav")

In [54]:
output

{'text': " he's been getting everyone of them spot on to start with we thought he was a bit late to come into batter number seven but then his whole idea was to stay till the end got an amazing 90 to bat with then bringing on the two CMs to finish off early bowling the 17th and the 18th and now a fresh ball to bowl the last over six first goal six was in the court for a shot that is Shiva Amtube that's the first one you'll need a few more of that you'll be just 26 might be too many maybe she won't do better than things though let's see this has the last over the six of the difference here he needs to keep going he needs three more",
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 2.0,
   'text': " he's been getting everyone of them",
   'tokens': [50364, 415, 311, 668, 1242, 1518, 295, 552, 50464],
   'temperature': 0.0,
   'avg_logprob': -0.393993971777744,
   'compression_ratio': 1.6451612903225807,
   'no_speech_prob': 0.7813397645950317},
  {'id': 1,
   'seek': 0,


In [9]:
import whisper

model = whisper.load_model("base")

def extract_text(vid: VideoFileClip, start: float, end: float):
    global model

    # Create subclip
    clip = vid.subclip(start, end)
    # Save the subclip
    path = f'{AUDIO_DIR}{str(int(start + end))}.mp3'
    clip.audio.write_audiofile(path)
    # Load the audio to be transcribed
    output = model.transcribe(path)
    return output

In [10]:
from multiprocessing import Lock
def extract_text(l: Lock, segments, audio, start, end):
    global model
    # try:
    # Save the subclip audio
    path = f"{AUDIO_DIR}{str(int(start + end))}.wav"
    audio[start*1000: end*1000].export(path, format="wav")

    # Transcribe audio
    output = {"text": "", "segments": []}
    # try:
    output = model.transcribe(path)
    # finally:
    # Add segments to the thread safe segment list
    for i in output["segments"]:
        l.acquire()
        try:
            segments.append({"text": i["text"], "start": i["start"], "end": i["end"]})
        finally:
            l.release()
    # finally:
    #     return

In [18]:
from multiprocessing.pool import ThreadPool
from pydub import AudioSegment
if __name__ == "__main__":
    # Get the video duration
    vid = VideoFileClip(SAMPLES_DIR + VIDEO_NAME)
    duration = vid.duration
    # Convert to audio
    video = AudioSegment.from_file(SAMPLES_DIR+VIDEO_NAME, format="mp4")
    audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)
    audio.export(SAMPLES_DIR+AUDIO_NAME, format="wav")
    audio = AudioSegment.from_file(SAMPLES_DIR+AUDIO_NAME, format="wav")

    try:
        segments = list()
        # Create a lock to append to the segments extracted
        sl = Lock()
        # Create the thread pool for extraction
        with ThreadPool() as pool:
            # No. of processes
            np = pool._processes
            # Chunk size
            chunk = duration / np
            # Excess time for overlaps
            excess = 5
            # Create partitions
            args = [[chunk * (i) - excess, chunk * (i + 1) + excess] for i in range(np)]
            args[0][0] += excess
            args[-1][1] -= excess
            print(args)

            pool.starmap(extract_text, [tuple([sl, segments, audio]) + tuple(i) for i in args])

        print(segments)
    finally:
        print()

[[0.0, 37.0325], [27.0325, 69.065], [59.065, 101.0975], [91.0975, 133.13], [123.13, 165.1625], [155.1625, 197.195], [187.195, 229.2275], [219.2275, 256.26]]



RuntimeError: The size of tensor a (9) must match the size of tensor b (3) at non-singleton dimension 3

In [19]:
args

[[0.0, 37.0325],
 [27.0325, 69.065],
 [59.065, 101.0975],
 [91.0975, 133.13],
 [123.13, 165.1625],
 [155.1625, 197.195],
 [187.195, 229.2275],
 [219.2275, 256.26]]

In [25]:
arg = [[i + 1] + args[i] for i in range(len(args))]

In [26]:
arg

[[1, 0.0, 37.0325],
 [2, 27.0325, 69.065],
 [3, 59.065, 101.0975],
 [4, 91.0975, 133.13],
 [5, 123.13, 165.1625],
 [6, 155.1625, 197.195],
 [7, 187.195, 229.2275],
 [8, 219.2275, 256.26]]

In [27]:
import pandas as pd

pd.DataFrame(data=arg, columns=["Chunk", "Start of chunk", "End of chunk"])

,Chunk,Start of chunk,End of chunk
0,1,0.0000,37.0325
1,2,27.0325,69.0650
2,3,59.0650,101.0975
3,4,91.0975,133.1300
4,5,123.1300,165.1625
5,6,155.1625,197.1950
6,7,187.1950,229.2275
7,8,219.2275,256.2600


In [12]:
args

[[32.0325, 69.065],
 [59.065, 101.0975],
 [91.0975, 133.13],
 [123.13, 165.1625],
 [155.1625, 197.195],
 [187.195, 229.2275],
 [219.2275, 256.26]]

In [63]:
import multiprocessing
import whisper

# Create a new process to handle ASR requests
asr_process = multiprocessing.Process(target=whisper.transcribe)

# Start the process
asr_process.start()

# Define a function to perform ASR
def perform_asr(audio):
    # Send the ASR request to the process
    asr_process.send(audio)
    result = asr_process.recv()
    # Return the result
    return result

# Use a thread pool to perform ASR
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor() as executor:
    # Load audio files (or chunks) into a list
    np = pool._processes
    # Chunk size
    chunk = duration/np
    # Excess time for overlaps
    excess = 5
    # Create partitions
    args = [[chunk*(i) - excess, chunk*(i + 1) + excess] for i in range(np)]
    args[0][0] += excess
    args[-1][1] -= excess
    audio_list = [f"{AUDIO_DIR}{str(int(i[0] + i[1]))}.wav" for i in args] # list of AudioSegment objects or audio file paths
    # Perform ASR on each audio file (or chunk)
    results = list(executor.map(perform_asr, audio_list))

# Stop the process
asr_process.terminate()

print(results)


AttributeError: 'Process' object has no attribute 'send'

Process Process-2:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/sportlight/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/miniconda3/envs/sportlight/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
TypeError: transcribe() missing 2 required positional arguments: 'model' and 'audio'


In [49]:
from multiprocessing.pool import ThreadPool

if __name__ == '__main__':
    # Create the thread pool
    with ThreadPool() as pool:
        # No. of processes
        np = pool._processes
        # Chunk size
        chunk = duration/np
        # Excess time for overlaps
        excess = 5
        # Create partitions
        args = [[chunk*(i) - excess, chunk*(i + 1) + excess] for i in range(np)]
        args[0][0] += excess
        args[-1][1] -= excess

        for result in pool.starmap(extract_text, [tuple([vid]) + tuple(i) for i in args]):
            print(result)
        

MoviePy - Writing audio in ../samples/audio/197.mp3


chunk:   0%|          | 0/4355 [00:00<?, ?it/s, now=None]




                                                                











                             












                             



chunk:   0%|          | 1/4355 [00:00<02:05, 34.72it/s, now=None]



























                                                                 











                             












                             



chunk:   0%|          | 1/4355 [00:00<02:42, 26.72it/s, now=None]



























                                                                 











                             












                             



chunk:   0%|          | 1/4355 [00:00<03:52, 18.73it/s, now=None]



























                                                                 











                             












                             



chunk:   0%|          | 1/4355 [00:00<05:12, 13.95it

MoviePy - Writing audio in ../samples/audio/962.mp3
MoviePy - Writing audio in ../samples/audio/2117.mp3
MoviePy - Writing audio in ../samples/audio/2502.mp3
MoviePy - Writing audio in ../samples/audio/577.mp3
MoviePy - Writing audio in ../samples/audio/2882.mp3













                                                                 
                                                      











                             



                                             












                             

                                                   


                                                



chunk:   0%|          | 2/4355 [00:00<03:50, 18.92it/s, now=None]

































                                                                 
                                                                 

MoviePy - Writing audio in ../samples/audio/1732.mp3














                             



                                                     












                             

                                                           


                                                        



chunk:   0%|          | 2/4355 [00:00<03:50, 18.92it/s, now=None]




























chunk:   0%|          | 2/4355 [00:00<03:50, 18.92it/s, now=None]

MoviePy - Writing audio in ../samples/audio/1347.mp3


chunk:   0%|          | 4/4355 [00:00<03:51, 18.81it/s, now=None]
































chunk:   0%|          | 7/4355 [00:00<03:19, 21.82it/s, now=None]








chunk:   1%|          | 29/4355 [00:00<00:48, 88.67it/s, now=None]




chunk:   1%|          | 49/4355 [00:00<00:35, 121.37it/s, now=None]




chunk:   2%|▏         | 79/4355 [00:00<00:25, 170.20it/s, now=None]









chunk:   3%|▎         | 135/4355 [00:00<00:19, 215.07it/s, now=None]




chunk:   4%|▎         | 157/4355 [00:00<00:19, 214.50it/s, now=None]




chunk:   4%|▍         | 189/4355 [00:01<00:18, 224.00it/s, now=None]




chunk:   5%|▌         | 221/4355 [00:01<00:17, 236.27it/s, now=None]









chunk:   6%|▌         | 251/4355 [00:01<00:17, 230.80it/s, now=None]




chunk:   7%|▋         | 288/4355 [00:01<00:16, 244.98it/s, now=None]




chunk:   7%|▋         | 319/4355 [00:01<00:16, 250.43it/s, now=None]




chunk:   8%|▊         | 349/4355 [00:01<00:15, 251.58it/s, now=None]




chunk:   9%|▉      

MoviePy - Done.


AttributeError: 'NoneType' object has no attribute 'wait'